<a href="https://colab.research.google.com/github/heitorcfelix/Agente-Revisor-de-Codigo/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai google-adk PyGithub

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from github import Github

# Authentication is defined via github.Auth
from github import Auth

# using an access token
auth = Auth.Token(userdata.get('GITHUB_API_KEY'))

# First create a Github instance:
g = Github(auth=auth)

# Close connection
g.close()

In [ ]:
# Criar dropdown para escolher um repositório
from ipywidgets import Dropdown, Output
import ipywidgets as widgets
from IPython.display import display, clear_output

# Variável global para armazenar o repositório atual
global repo
global repo_selected

# Inicializar variável de controle
repo_selected = False

# Obter lista de repositórios do usuário
repos = g.get_user().get_repos()
repo_names = [repo.name for repo in repos]

# Criar dropdown para seleção de repositório
repo_dropdown = Dropdown(
    description='Escolha um repositório:',
    options=repo_names,
    value=None,  # Inicialmente sem valor selecionado
    layout={'width': 'max-content'}
)

output = Output()

def on_repo_change(change):
    global repo, repo_selected
    with output:
        output.clear_output()
        # Acessar o repositório selecionado
        repo = g.get_user().get_repo(change.new)
        print(f"Repositório selecionado: {repo.name}")

        # Tentar obter e exibir o README
        try:
            readme = repo.get_contents("README.md")
            display(to_markdown(readme.decoded_content.decode("utf-8")))
        except:
            print("README.md não encontrado ou não pode ser exibido.")

        # Marcar que um repositório foi selecionado
        repo_selected = True

repo_dropdown.observe(on_repo_change, names='value')

# Exibir o dropdown e a área de saída
display(repo_dropdown)
display(output)

# Verificar se há repositórios disponíveis
if not repo_names:
    print("Nenhum repositório encontrado.")
else:
    print("Por favor, selecione um repositório para continuar.")


In [ ]:
def _listar_todos_os_arquivos(repo, path="/"):
    arquivos = []
    conteudo = repo.get_contents(path)
    while conteudo:
        arquivo = conteudo.pop(0)
        if arquivo.type == "dir":
            conteudo.extend(repo.get_contents(arquivo.path))
        else:
            arquivos.append(arquivo)
    return arquivos


In [ ]:
##########################################
# --- Agente 1: Agente interpletador de arquivo de código --- #
##########################################
def agente_leitor(arquivo):

    analista_codigo = Agent(
        name="analista_codigo",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de análise de código. Sua tarefa é analisar **um arquivo de código por vez**, recebido como entrada,
        e gerar uma explicação clara e objetiva sobre o que esse arquivo faz. A explicação não deve conter o conteúdo no arquivo, apenas o objetibo geral do arquivo.

        Para cada arquivo que receber:
        - Indique o nome do arquivo.
        - Explique qual é o propósito geral do arquivo dentro de um projeto de software.
        - Destaque e explique as principais funções, classes, componentes ou blocos de lógica presentes.
        - Se possível, comente como esse arquivo pode se relacionar com outras partes de um sistema típico (ex: "essa função pode ser usada para autenticar usuários").

        O conteúdo será passado um arquivo por vez. Não é necessário lidar com múltiplos arquivos ou navegar por diretórios.

        Ignore arquivos que não sejam código-fonte (por exemplo: imagens, arquivos binários, .md, .txt, etc.) ou que não contenham lógica significativa.

        O objetivo é ajudar alguém a entender, de forma didática, o papel e funcionamento de um único arquivo de código dentro de um sistema.
        """,
        description="Agente que analisa e explica um arquivo de código por vez.",
    )


    analise = call_agent(analista_codigo, arquivo)
    return analise

In [ ]:
def analisar_repositorio(repo, path="/"):
    all_analyses = []

    def _processar_arquivo(path):
        contents = repo.get_contents(path)

        # Handle if contents is a single file
        if not isinstance(contents, list):
            contents = [contents]

        for item in contents:
            if item.type == 'file':
                print(f"Analisando: {item.path}")
                try:
                    conteudo = repo.get_contents(item.path).decoded_content.decode("utf-8")
                    resposta = agente_leitor(conteudo)
                    all_analyses.append(f"## {item.path}\n\n{resposta}")
                except Exception as e:
                    all_analyses.append(f"## {item.path}\n\nErro ao analisar: {str(e)}")
            elif item.type == 'dir':
                _processar_arquivo(item.path)

    _processar_arquivo(path)
    return "\n\n---\n\n".join(all_analyses)

# Analisar todo o repositório recursivamente
print("Iniciando análise recursiva do repositório...")
resultado_completo = analisar_repositorio(repo)
print("Análise concluída. Resultado completo:\n")
display(to_markdown(resultado_completo))

In [ ]:
##########################################
# --- Agente 2: Agente analista de projetos --- #
##########################################
def agente_analista(descricao_codigo):
    avaliador_projeto = Agent(
        name="avaliador_projeto",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em análise de projetos de software. Sua tarefa é analisar a estrutura e os componentes
        de um projeto de código com base em uma listagem em Markdown contendo:

        - O nome de cada arquivo no projeto.
        - Uma breve descrição do conteúdo ou função de cada arquivo.

        Com base nessas informações, forneça uma **avaliação geral do projeto**, seguindo o seguinte formato:

        1. **Descrição geral do projeto**:
        Explique o que o projeto parece fazer, qual é o seu propósito, e para que tipo de aplicação ele é voltado.

        2. **Pontos fortes**:
        Liste aspectos positivos como organização da estrutura, separação de responsabilidades, clareza nos nomes dos arquivos,
        presença de testes, documentação, boas práticas visíveis, modularização etc.

        3. **Pontos fracos**:
        Aponte aspectos que possam ser melhorados, como ausência de testes, acoplamento excessivo, falta de estrutura de pastas,
        arquivos muito genéricos ou confusos, ausência de padrões claros etc.

        4. **Recomendações e próximos passos**:
        Sugira o que deveria ser feito para evoluir o projeto — como reorganização, documentação adicional, melhorias técnicas,
        refatoração, adição de testes, melhorias de segurança, integração contínua, etc.

        Use um tom analítico e profissional, como um revisor técnico que deseja ajudar a melhorar o projeto.

        O conteúdo será fornecido em formato Markdown, com a listagem dos arquivos e suas descrições.
        Não assuma nada além do que está presente nesse conteúdo.
        """,
        description="Agente que avalia a qualidade geral de um projeto de software com base em uma listagem de arquivos e descrições.",
    )

    analise = call_agent(avaliador_projeto, descricao_codigo)
    return analise

In [ ]:
resultado = agente_analista(resultado_completo)
print("\n--- 📝 Resultado do Agente 2 (Analista) ---\n")
display(to_markdown(resultado))
print("--------------------------------------------------------------")

In [ ]:
# criar uma branch chamada evaluation, adicione o resultado em um arquivo evaluation.md e faça um PR para branch main

# Define o nome da branch e do arquivo de avaliação
branch_name = "evaluation"
evaluation_filename = "evaluation.md"

# Obtém o usuário autenticado
user = g.get_user()

# Obtém o repositório novamente (para garantir que temos o objeto mais recente)
# Assumindo que 'repo' já está definido no código anterior como o objeto do repositório
# repo = user.get_repo("OpenWeatherService") # Descomente e ajuste o nome do repo se necessário

try:
    # Verifica se a branch de avaliação já existe
    try:
        ref = repo.get_git_ref(f"heads/{branch_name}")
        print(f"Branch '{branch_name}' já existe. Atualizando...")
    except Exception as e:
        # Se a branch não existe, cria uma nova a partir da branch principal (main)
        print(f"Branch '{branch_name}' não encontrada. Criando nova branch...")
        main_branch = repo.get_branch("main")
        repo.create_git_ref(ref=f"refs/heads/{branch_name}", sha=main_branch.commit.sha)
        print(f"Branch '{branch_name}' criada com sucesso.")
        ref = repo.get_git_ref(f"heads/{branch_name}") # Obtém a referência da nova branch

    # Verifica se o arquivo evaluation.md já existe na branch de avaliação
    try:
        contents = repo.get_contents(evaluation_filename, ref=branch_name)
        # Atualiza o arquivo existente
        print(f"Arquivo '{evaluation_filename}' encontrado na branch '{branch_name}'. Atualizando...")
        repo.update_file(
            path=evaluation_filename,
            message=f"Atualiza a avaliação do projeto",
            content=resultado,
            sha=contents.sha,
            branch=branch_name
        )
        print(f"Arquivo '{evaluation_filename}' atualizado com sucesso.")
    except Exception as e:
         # Se o arquivo não existe, cria um novo
        print(f"Arquivo '{evaluation_filename}' não encontrado na branch '{branch_name}'. Criando novo arquivo...")
        repo.create_file(
            path=evaluation_filename,
            message=f"Adiciona a avaliação inicial do projeto",
            content=resultado,
            branch=branch_name
        )
        print(f"Arquivo '{evaluation_filename}' criado com sucesso.")

    # Cria o Pull Request (PR)
    print(f"Criando Pull Request da branch '{branch_name}' para 'main'...")
    try:
        pr = repo.create_pull(
            title=f"Avaliação automática do projeto",
            body=f"Este Pull Request adiciona/atualiza o arquivo `{evaluation_filename}`.",
            head=branch_name,
            base="main"
        )
        print(f"Pull Request criado com sucesso: {pr.html_url}")
    except Exception as e:
        print(f"Erro ao criar o Pull Request: {e}")
        # Verifica se já existe um PR da mesma branch para base
        open_prs = repo.get_pulls(state='open', head=f'{user.login}:{branch_name}', base='main')
        if open_prs.totalCount > 0:
            print("Já existe um Pull Request aberto desta branch para 'main'.")
            print(f"PR existente: {open_prs[0].html_url}")
        else:
             print("Ocorreu um erro inesperado ao criar o PR.")


except Exception as e:
    print(f"Ocorreu um erro geral: {e}")

# Fechar a conexão do Github ao final
g.close()


In [ ]:
##########################################
# --- Agente 3: Assistente de códigos --- #
##########################################
def agente_programador(codigo):
    assistente_codigo = Agent(
        name="assistente_codigo",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de melhoria de código. Receberá como entrada um único arquivo de código (como .py ou .ipynb convertido para texto)
        e deve retornar uma **versão aprimorada do código original**.

        Seu objetivo é aplicar melhorias com base em:

        - Boas práticas de Clean Code
        - Comentários úteis e objetivos onde fizer sentido
        - Docstrings explicativas em classes e funções, se necessário
        - Nomes de variáveis e funções mais descritivos (evitar nomes genéricos)
        - Estilo de formatação consistente (ex: PEP8 para Python)

        **IMPORTANTE:**
        - A saída deve conter **somente o código Python melhorado**.
        - Não inclua qualquer explicação, justificativa ou comentários fora do código.
        - Não use blocos Markdown (ex: ```python).
        - Não escreva frases como “Melhorias sugeridas” ou “Código atualizado”.
        - Apenas retorne o código limpo, direto, pronto para ser usado.

        ### Exemplo

        #### Entrada:
        class Weather(db.Model):
            user_id = db.Column(db.Integer, primary_key=True)
            date_time = db.Column(db.DateTime)
            json_data = db.Column(db.JSON)

        #### Saída esperada:
        class Weather(db.Model):
            '''
            Modelo para armazenar dados meteorológicos.

            Atributos:
                user_id (int): Identificador do usuário.
                date_time (datetime): Data e hora da coleta.
                json_data (JSON): Dados meteorológicos coletados.
            '''
            user_id = db.Column(db.Integer, primary_key=True)
            date_time = db.Column(db.DateTime)
            json_data = db.Column(db.JSON)
        """,
        description="Agente que reescreve código Python (ou notebooks) aplicando boas práticas — saída somente com o código puro.",
    )

    analise = call_agent(assistente_codigo, codigo)
    return analise

In [ ]:
from ipywidgets import Dropdown, Output, Button
from IPython.display import Markdown, display

# Variável global para armazenar as sugestões do agente
ultima_sugestao_agente = None

# Obter a lista de arquivos do repositório
arquivos_repo = _listar_todos_os_arquivos(repo)
# Criar uma lista de nomes de arquivos para o dropdown
nomes_arquivos = [arq.path for arq in arquivos_repo]

# Criar o Dropdown
dropdown_arquivos = Dropdown(
    options=nomes_arquivos,
    description='Escolha um arquivo:',
    disabled=False,
)

# Área de saída para exibir o resultado
output_area = Output()

# Função para lidar com a seleção do Dropdown
def on_value_change(change):
    global ultima_sugestao_agente

    with output_area:
        output_area.clear_output()
        selected_file_path = change['new']
        print(f"Arquivo selecionado: {selected_file_path}")

        try:
            # Obter o conteúdo do arquivo selecionado
            file_content = repo.get_contents(selected_file_path).decoded_content.decode("utf-8")
            print("Conteúdo do arquivo lido.")

            # Submeter o conteúdo para o agente_programador
            print("Submetendo conteúdo para o agente_programador...")
            sugestoes_agente = agente_programador(file_content)
            ultima_sugestao_agente = sugestoes_agente  # Armazenar na variável global
            print("Sugestões finalizadas pelo agente programador")
            print(sugestoes_agente)

        except Exception as e:
            display(Markdown(f"Ocorreu um erro ao processar o arquivo: {e}"))
            ultima_sugestao_agente = None

# Registrar a função para ser chamada quando o valor do Dropdown mudar
dropdown_arquivos.observe(on_value_change, names='value')

# Exibir o Dropdown e a área de saída
display(dropdown_arquivos, output_area)

# Função para obter a última sugestão do agente
def obter_ultima_sugestao():
    return ultima_sugestao_agente


In [ ]:
# Função para criar um Pull Request com a última sugestão do agente
def criar_pull_request():
    global ultima_sugestao_agente

    if ultima_sugestao_agente is None:
        display(Markdown("Não há sugestões disponíveis. Por favor, selecione um arquivo primeiro."))
        return

    try:
        # Obter o arquivo selecionado
        selected_file_path = dropdown_arquivos.value

        # Obter o conteúdo atual do arquivo
        file_content = repo.get_contents(selected_file_path).decoded_content.decode("utf-8")

        # Aplicar a sugestão do agente ao conteúdo do arquivo
        novo_conteudo = ultima_sugestao_agente

        # Criar um novo branch
        branch_name = f"sugestao-agente"
        source_branch = repo.get_branch("main")  # ou master, dependendo do repositório
        repo.create_git_ref(ref=f"refs/heads/{branch_name}", sha=source_branch.commit.sha)

        # Fazer commit da alteração no novo branch
        commit_message = "Aplicar sugestão do agente programador"
        repo.update_file(
            path=selected_file_path,
            message=commit_message,
            content=novo_conteudo,
            sha=repo.get_contents(selected_file_path, ref=branch_name).sha,
            branch=branch_name
        )

        # Criar o Pull Request
        pr = repo.create_pull(
            title=f"Sugestão do agente para {selected_file_path}",
            body="Este PR contém sugestões geradas automaticamente pelo agente programador.",
            head=branch_name,
            base="main"  # ou master, dependendo do repositório
        )

        display(Markdown(f"✅ Pull Request criado com sucesso: [PR #{pr.number}]({pr.html_url})"))

    except Exception as e:
        display(Markdown(f"❌ Erro ao criar Pull Request: {str(e)}"))

# Botão para criar o Pull Request
botao_criar_pr = Button(
    description='Criar Pull Request',
    button_style='success',
    tooltip='Criar um PR com a última sugestão do agente'
)

# Conectar o botão à função
botao_criar_pr.on_click(lambda b: criar_pull_request())

# Exibir o botão
display(botao_criar_pr)
